Attempt 1

In [2]:
# !pip install --update pyldpc
import pyldpc
import numpy as np

In [2]:
n = 15  # Number of columns
d_v = 4 # Number of ones per column, must be lower than d_c (because H must have more rows than columns)
d_c = 5 # Number of ones per row, must divide n (because if H has m rows: m*d_c = n*d_v (compute number of ones in H))

H = pyldpc.RegularH(n,d_v,d_c)

print("Regular parity-check matrix H({},{},{}):\n\n".format(n,d_v,d_c),H)

Regular parity-check matrix H(15,4,5):

 [[1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1]
 [0 1 0 1 0 0 1 0 1 1 0 0 0 0 0]
 [0 0 1 0 1 0 0 0 0 0 0 1 0 1 1]
 [1 0 0 0 0 1 0 1 0 0 1 0 1 0 0]
 [0 1 0 0 1 0 0 0 1 1 1 0 0 0 0]
 [0 0 0 1 0 0 0 1 0 0 0 1 1 0 1]
 [1 0 1 0 0 1 1 0 0 0 0 0 0 1 0]
 [1 0 0 0 0 0 1 0 0 1 0 1 0 1 0]
 [0 1 0 0 1 1 0 1 1 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0 0 0 1 0 1 0 1]]


In [3]:
tG = pyldpc.CodingMatrix(H)
print("Transposed Coding Matrix tG that goes with H above is:\n\n",tG)
print("\n With G,H you can code messages of {} bits into codewords of {} bits because G's shape is {}\n".format(tG.shape[1], tG.shape[0],tG.T.shape))

Transposed Coding Matrix tG that goes with H above is:

 [[1 0 0 1 0 0]
 [1 1 0 1 0 1]
 [0 1 0 0 0 0]
 [0 1 1 0 1 0]
 [0 1 1 0 1 1]
 [1 1 1 0 0 0]
 [0 0 1 1 1 0]
 [0 1 0 1 1 1]
 [0 0 0 0 0 1]
 [1 0 0 0 0 0]
 [0 0 1 1 1 1]
 [0 0 1 0 0 0]
 [0 0 0 1 0 0]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]]

 With G,H you can code messages of 6 bits into codewords of 15 bits because G's shape is (6, 15)



Attempt 2

In [1]:
# input
import numpy as np
import scipy
import scipy.sparse
from scipy.sparse import csr_matrix, csc_matrix

H = np.array([[1, 1, 0, 1, 0, 0], 
              [0, 1, 1, 0, 1, 0],
              [1, 0, 0, 0, 1, 1],
              [0, 0, 1, 1, 0, 1]])

c = np.array([0, 0, 1, 0, 1, 1])
y = np.array([1, 0, 1, 0, 1, 1])
p = 0.2

In [2]:
log_p = np.log(p/(1-p))
r = log_p * (2 * (y == 1) - 1)
r

array([-1.38629436,  1.38629436, -1.38629436,  1.38629436, -1.38629436,
       -1.38629436])

In [3]:
def simple_sum_product_decording(H, r, maxiter=1):
#     Prepare
    n_check_nodes, n_bit_nodes = H.shape
    H_csr = csr_matrix(H)
    H_csc = H_csr.tocsc(copy=True)
    
    csr_ptr = H_csr.indptr
    csr_ind = H_csr.indices
    
    csc_ptr = H_csc.indptr
    csc_ind = H_csc.indices
    
    nnz = H_csr.nnz
    M_csc = np.empty(nnz, dtype=np.float64)
    E_csr = np.empty(nnz, dtype=np.float64)
    L = np.empty(n_bit_nodes, dtype=np.float64)
#     Initialize
    for i in range(n_bit_nodes):
#         M[csr_ptr[k]:csr_ptr[k+1]] = r[csr_ind[csr_ptr[k]:csr_ptr[k+1]]]
        M_csc[csc_ptr[i]:csc_ptr[i+1]] = r[i]
#         M_csc[csc_ptr[k]:csc_ptr[k+1]] = r[csc_ind[csc_ptr[k]:csc_ptr[k+1]]]
    print("M:\n", csc_matrix((M_csc, csc_ind, csc_ptr)).toarray())
    
    for ii in range(maxiter):
        M_csr = csc_matrix((M_csc, csc_ind, csc_ptr)).tocsr().data
        for j in range(n_check_nodes):
            j_beg = csr_ptr[j]
            j_end = csr_ptr[j+1]
            M_local = np.tanh(M_csr[j_beg:j_end] / 2)
            for e in range(j_end-j_beg):
                prod_local = M_local[:e].prod() * M_local[e+1:].prod()
                E_csr[j_beg+e] = np.log((1. + prod_local) / (1. - prod_local))
        
        print("E:\n", csr_matrix((E_csr, csr_ind, csr_ptr)).toarray())
        E_csc = csr_matrix((E_csr, csr_ind, csr_ptr)).tocsc().data
#         Get message
        L = np.array(r)
        for i in range(n_bit_nodes):
            L[i] += E_csc[csc_ptr[i]:csc_ptr[i+1]].sum()
        z = 1 * (L <= 0)
#         Check message  
        if np.any(H.dot(z) % 2 != 0):
#             Update M
            for i in range(n_bit_nodes):
                i_beg = csc_ptr[i]              #column beg
                i_end = csc_ptr[i+1]            #column end
                E_local = E_csc[i_beg:i_end]
                for e, j in range(csc_ind[i_beg:i_end]):
                    M[i_beg+e] = E_local[:e].sum() + E_local[e+1:].sum() + r[i]
        else:
            break
    return z
        

In [4]:
simple_sum_product_decording(H, r)

M:
 [[-1.38629436  1.38629436  0.          1.38629436  0.          0.        ]
 [ 0.          1.38629436 -1.38629436  0.         -1.38629436  0.        ]
 [-1.38629436  0.          0.          0.         -1.38629436 -1.38629436]
 [ 0.          0.         -1.38629436  1.38629436  0.         -1.38629436]]
E:
 [[ 0.7537718 -0.7537718  0.        -0.7537718  0.         0.       ]
 [ 0.         0.7537718 -0.7537718  0.        -0.7537718  0.       ]
 [ 0.7537718  0.         0.         0.         0.7537718  0.7537718]
 [ 0.         0.        -0.7537718  0.7537718  0.        -0.7537718]]


array([0, 0, 1, 0, 1, 1])